In [1]:
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

     |████████████████████████████████| 3.1 MB 21.4 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 26.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 23.1 MB/s eta 0:00:01
     |████████████████████████████████| 19.2 MB 23.7 MB/s eta 0:00:01     |████████████████████████████████| 19.1 MB 23.7 MB/s eta 0:00:01
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314478 sha256=55efb52c6a4b2bbef62efbe8a22c702b22196ed5785a2351f79907185c38168c
  Stored in directory: /root/.cache/pip/wheels/ca/38/d8/dfdfe73bec1d12026b30cb7ce8da650f3f0ea2cf155ea018ae
Successfully built jieba
     |████████████████████████████████| 108.2 MB 121 kB/s  eta 0:00:01
     |████████████████████████████████| 3.1 MB 28.7 MB/s eta 0:00:01


In [1]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import FusionEmbedding,GlyphEmbedding,PinyinEmbedding
from model.fusionDataset import FusionDataset

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Example for glyph + pinyin embedding

In [ ]:
glyph = GlyphEmbedding('data/glyph_weight.npy')
pinyinV = PinyinEmbedding(30,'data/pinyin_map.json')
# Example
display(glyph(torch.tensor([0,1,2])))
display(pinyinV(torch.tensor([0,1])))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0421, -0.0327,  0.0096,  ...,  0.0711,  0.0417, -0.0224],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward>)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.2590,  0.2719,  0.1614,  0.0099,  0.1555, -0.2728,  0.2700, -0.1006,
         -0.0885,  0.0607,  0.2121,  0.1352,  0.1809, -0.0842,  0.2739, -0.2115,
         -0.2756, -0.1202,  0.0751, -0.2282, -0.2437, -0.2295,  0.2278, -0.2172,
          0.0318, -0.2727,  0.0463, -0.2532, -0.1979,  0.1673]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
# Example
_ = tokenizer('[UNK]辛业怠π[PAD]',return_tensors='pt')
display(_)
Anchibert(**_).last_hidden_state.shape

{'input_ids': tensor([[ 101,  100, 6789,  689, 2591,  223,    0,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

torch.Size([1, 8, 768])

In [ ]:
tokenizer.convert_ids_to_tokens(_['input_ids'][0,:])

['[CLS]', '[UNK]', '辛', '业', '怠', 'π', '[PAD]', '[SEP]']

### Load Necessary preproceeded Data

In [2]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [4]:
trainSet = FusionDataset(tr_in[:110],tokenizer,glyph2ix,pinyin2ix,pos2ix,tr_out[:110],tr_pos_in[:110],tr_pos_out[:110])

110it [00:00, 2153.05it/s]
110it [00:00, 2263.32it/s]


In [5]:
trainLoader = DataLoader(trainSet,batch_size=64,shuffle=True)
#     validLoader = DataLoader(validSet,batch_size=batch_size,shuffle=False)

In [6]:
from model.fusion_transformer import Fusion_Anchi_Trans_Decoder, Fusion_Anchi_Transformer

In [7]:
config = { # for Fusion_Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30,
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6,
    'output_dim':21128 # fixed
}
config2 = { # Fusion_Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30,
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_encoder_layers':6,
    'num_decoder_layers':6,
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1
}

In [20]:
# model = Fusion_Anchi_Trans_Decoder(config).to(device)

In [8]:
model2 = Fusion_Anchi_Transformer(config2).to(device)

In [ ]:
# model3 = FusionEmbedding(config)
# model3.to(device)

FusionEmbedding(
  (position_embeddings): Embedding(50, 768)
  (glyph_embeddings): GlyphEmbedding(
    (embedding): Embedding(9110, 576)
  )
  (pinyin_embeddings): PinyinEmbedding(
    (embedding): Embedding(1297, 30, padding_idx=0)
  )
  (pos_tag_embeddings): Embedding(30, 10, padding_idx=0)
  (fc): Linear(in_features=1384, out_features=768, bias=True)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
criterion = nn.NLLLoss()

In [18]:
import torch.optim as optim

In [19]:
optimizer = optim.Adam(model2.parameters(),lr=0.01)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
Anchibert.to(device)
for Xsents_input_ids,Xsents_token_type_ids,\
    Xsents_attention_mask,Xsents_pinyin_ids,\
    Xsents_glyph_ids,Xsents_pos_ids,\
    Ysents_input_ids,Ysents_token_type_ids,\
    Ysents_attention_mask,Ysents_pinyin_ids,\
    Ysents_glyph_ids,Ysents_pos_ids in trainLoader:
    
    Xsents_input_ids = Xsents_input_ids.to(device)
    Xsents_token_type_ids = Xsents_token_type_ids.to(device)
    Xsents_attention_mask = Xsents_attention_mask.to(device)
    Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
    Xsents_glyph_ids = Xsents_glyph_ids.to(device)
    Xsents_pos_ids = Xsents_pos_ids.to(device)
    Ysents_input_ids = Ysents_input_ids.to(device)
    Ysents_token_type_ids = Ysents_token_type_ids.to(device)
    Ysents_attention_mask = Ysents_attention_mask.to(device)
    Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
    Ysents_glyph_ids = Ysents_glyph_ids.to(device)
    Ysents_pos_ids = Ysents_pos_ids.to(device)
    
    Xword_embeddings = Anchibert(Xsents_input_ids,      \
                                 Xsents_token_type_ids, \
                                 Xsents_attention_mask  \
                                )['last_hidden_state'].detach()


    Yword_embeddings = Anchibert(Ysents_input_ids,      \
                                 Ysents_token_type_ids, \
                                 Ysents_attention_mask  \
                                )['last_hidden_state'].detach()
    # outputs1 for Fusion_Anchi_Trans_Decoder
#     outputs1 = model(Xword_embeddings,Xsents_pinyin_ids, \
#                     Xsents_token_type_ids,Xsents_pos_ids,\
#                     Yword_embeddings,Ysents_pinyin_ids, \
#                     Ysents_token_type_ids,Ysents_pos_ids,\
#                     Xsents_attention_mask.bool(),Ysents_attention_mask.bool())
    
    # outputs2 Fusion_Anchi_Transformer(config2)
    outputs2= model2(Xword_embeddings,Xsents_pinyin_ids, \
                    Xsents_token_type_ids,Xsents_pos_ids,\
                    Yword_embeddings,Ysents_pinyin_ids, \
                    Ysents_token_type_ids,Ysents_pos_ids,\
                    Xsents_attention_mask.bool(),Ysents_attention_mask.bool())
    
    # FusionEmbedding
#     outputembeddingsX = model3(Xword_embeddings,Xsents_pinyin_ids, \
#                          Xsents_token_type_ids,Xsents_pos_ids)
#     outputembeddingsY = model3(Yword_embeddings,Ysents_pinyin_ids, \
#                              Ysents_token_type_ids,Ysents_pos_ids)
    
    
    # get the true model output label from decoder input id
    trueY = torch.zeros(Ysents_input_ids.shape,dtype=torch.long,device=device)
    trueY[:,:-1] = Ysents_input_ids[:,1:]
    # find the index of first padding return 0 if no_padding 
    indices = torch.argmin(Ysents_attention_mask,dim=1).tolist()

    for i, index in enumerate(indices):
        trueY[i,index-1] = 102
    trueY = trueY.view(-1)
    
    outputs2 = outputs2.view(-1,outputs2.shape[-1])    
    loss = criterion(outputs2,trueY)
    loss.backward()
    break

In [ ]:
outputs.shape, outputs2.shape, outputembeddingsX.shape,outputembeddingsY.shape

torch.Size([28, 64, 21128])

### Way to generate new decoder input from raw sentence 

记得把model output 中 不存在 glyph dict的词以及'[SEP]' 和'[PAD]'给转换成 '_'。

In [21]:
sents = tr_out[:2]
sents

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['晨', '露', '润', '花', '花', '更', '红'], ['万', '方', '乐', '奏', '有', '于', '阗']]

In [24]:
FusionDataset.prepare_sequence(sents=sents, tokenizer=tokenizer,
                               glyph2ix=glyph2ix,pinyin2ix=pinyin2ix,
                               pos2ix=pos2ix,encode=False,skip_error=False)

Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
2it [00:00, 83.52it/s]


([tensor([ 101, 3247, 7463, 3883, 5709, 5709, 3291, 5273], dtype=torch.int32),
  tensor([ 101,  674, 3175,  727, 1941, 3300,  754,  100], dtype=torch.int32)],
 [tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32),
  tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)],
 [tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32),
  tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)],
 [tensor([  0, 717,  59, 873, 579, 579, 217, 334], dtype=torch.int32),
  tensor([   0,  187,  476,  501,  807, 1178,  263, 1002], dtype=torch.int32)],
 [tensor([   0, 5656, 1568, 3172, 3915, 3915, 3367, 8111], dtype=torch.int32),
  tensor([   0, 1729, 3171, 4526, 2058, 8413, 3025, 6866], dtype=torch.int32)],
 [tensor([ 0,  4,  4,  4,  4,  4, 16, 12], dtype=torch.int32),
  tensor([ 0, 26, 26, 26, 26,  9, 27, 27], dtype=torch.int32)])